CSV base

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import json
import re

Separar RUT de Nombre

In [2]:
def separate_profesor_data():
    # Read the CSV file with semicolon delimiter
    df = pd.read_csv('Guia_FIA.csv', delimiter=';', encoding='utf-8')
    
    # Create new columns for RUT and Professor name
    df['RUT'] = ''
    df['Profesor_Nombre'] = ''
    
    # Process only rows where 'Profesor' column is not empty and contains a pattern matching a RUT
    mask = df['Profesor'].notna() & df['Profesor'].str.contains(r'\d+-', na=False)
    
    for idx in df[mask].index:
        profesor_str = df.at[idx, 'Profesor']
        # Split by first space after the RUT pattern
        parts = profesor_str.split(' ', 1)
        if len(parts) == 2:
            df.at[idx, 'RUT'] = parts[0]
            df.at[idx, 'Profesor_Nombre'] = parts[1]
    
    # Drop the original 'Profesor' column
    df = df.drop('Profesor', axis=1)
    
    # Reorder columns to put RUT and Profesor_Nombre after the fifth column
    cols = df.columns.tolist()
    # Ensure RUT and Profesor_Nombre are only inserted once
    cols = cols[:5] + ['RUT', 'Profesor_Nombre'] + [col for col in cols[5:] if col not in ['RUT', 'Profesor_Nombre']]
    df = df[cols]
    
    # Write the modified DataFrame to a new CSV file
    df.to_csv('outputN1_Guia_FIA.csv', sep=';', index=False, encoding='utf-8')
    
    return df

if __name__ == "__main__":
    result_df = separate_profesor_data()
    print("CSV file has been processed successfully.")

CSV file has been processed successfully.


Separar Capacidad de Inscritos

In [3]:
def capacity_and_enrolled():
    # Read the output from the previous step
    df = pd.read_csv('outputN1_Guia_FIA.csv', delimiter=';', encoding='utf-8')
    
    # Create new columns
    df['Vacantes'] = ''
    df['Inscritos'] = ''
    
    # Process only rows where 'Dic (Cupo/Insc)' column is not empty
    mask = df['Dic (Cupo/Insc)'].notna()
    
    """
    for idx in df[mask].index:
        cupo_str = df.at[idx, 'Dic (Cupo/Insc)']
        # Extract numbers between parentheses
        if 'Nor (' in cupo_str:
            numbers = cupo_str.split('(')[1].split(')')[0]
            if '/' in numbers:
                vacantes, inscritos = numbers.split('/')
                df.at[idx, 'Vacantes'] = int(vacantes)
                df.at[idx, 'Inscritos'] = int(inscritos) """
                
    import re

    for idx in df[mask].index:
        cupo_str = df.at[idx, 'Dic (Cupo/Insc)']
        # Use regex to extract numbers
        match = re.search(r'\((\d+)/(\d+)\)', cupo_str)
        if match:
            vacantes = int(match.group(1))
            inscritos = int(match.group(2))
            df.at[idx, 'Vacantes'] = vacantes
            df.at[idx, 'Inscritos'] = inscritos
    
    # Drop the original column
    df = df.drop('Dic (Cupo/Insc)', axis=1)
    
    # Reorder columns to put Vacantes and Inscritos after the second column
    cols = df.columns.tolist()
    vacantes_index = cols.index('Vacantes')
    inscritos_index = cols.index('Inscritos')
    cols = cols[:2] + ['Vacantes', 'Inscritos'] + [col for col in cols[2:] if col not in ['Vacantes', 'Inscritos']]
    df = df[cols]
    
    # Write the modified DataFrame to a new CSV file
    df.to_csv('outputN2_Guia_FIA.csv', sep=';', index=False, encoding='utf-8')
    
    return df

# Execute the function
if __name__ == "__main__":
    result_df = capacity_and_enrolled()
    print("CSV file has been processed successfully.")
    # Print first few rows to verify

CSV file has been processed successfully.


Separar NombreSala y Capacidad

In [4]:
def separate_classroom_data():
    # Read the output from the previous step
    df = pd.read_csv('outputN2_Guia_FIA.csv', delimiter=';', encoding='utf-8')
    
    # Create new columns
    df['Sala'] = ''
    df['Capacidad'] = ''
    
    # Process only rows where 'Sala(Cap)' column is not empty
    mask = df['Sala(Cap)'].notna()
    
    for idx in df[mask].index:
        sala_str = df.at[idx, 'Sala(Cap)']
        if '(' in sala_str and ')' in sala_str:
            # Split into sala and capacidad
            sala = sala_str.split('(')[0]
            capacidad = sala_str.split('(')[1].split(')')[0]
            df.at[idx, 'Sala'] = sala
            if capacidad.isdigit():  # Only convert to int if it's a number
                df.at[idx, 'Capacidad'] = int(capacidad)
    
    # Drop the original column
    df = df.drop('Sala(Cap)', axis=1)
    
    # Reorder columns to put Sala and Capacidad where Sala(Cap) was
    cols = df.columns.tolist()
    cols = [col for col in cols if col not in ['Sala', 'Capacidad']]
    sala_cap_position = 10  # La posición mencionada (11) menos 1 por el índice base 0
    cols = cols[:sala_cap_position] + ['Sala', 'Capacidad'] + cols[sala_cap_position:]
    df = df[cols]
    
    # Write the modified DataFrame to a new CSV file
    df.to_csv('outputN3_Guia_FIA.csv', sep=';', index=False, encoding='utf-8')
    
    return df

# Execute the function
if __name__ == "__main__":
    result_df = separate_classroom_data()
    print("CSV file has been processed successfully.")

CSV file has been processed successfully.


Horas Impartidas

In [5]:
def obtain_hours():
    # Read the output from the previous step
    df = pd.read_csv('outputN3_Guia_FIA.csv', delimiter=';', encoding='utf-8')
    
    # Create new column for hours
    df['Cantidad Horas'] = 0
    
    # Process only rows where 'Detalle Hora' column is not empty
    mask = df['Detalle Hora'].notna()
    
    def calculate_hours(time_range):
        try:
            # Split the time range and extract start and end times
            start_str, end_str = time_range.split(' - ')
            
            # Convert times to datetime objects for easier calculation
            start_time = datetime.strptime(start_str, '%H:%M')
            end_time = datetime.strptime(end_str, '%H:%M')
            
            # Calculate the difference in hours
            diff = end_time - start_time
            hours = diff.seconds / 3600
            
            return int(hours)
        except:
            return 0
    
    # Apply the calculation to each row with time data
    df.loc[mask, 'Cantidad Horas'] = df.loc[mask, 'Detalle Hora'].apply(calculate_hours)
    
    # Reorder columns to put Cantidad Horas after Detalle Hora
    cols = df.columns.tolist()
    detalle_hora_index = cols.index('Detalle Hora')
    cols = [col for col in cols if col != 'Cantidad Horas']
    cols.insert(detalle_hora_index + 1, 'Cantidad Horas')
    df = df[cols]
    
    # Write the modified DataFrame to a new CSV file
    df.to_csv('outputN4_Guia_FIA.csv', sep=';', index=False, encoding='utf-8')
    
    return df

# Execute the function
if __name__ == "__main__":
    result_df = obtain_hours()
    print("CSV file has been processed successfully.")

CSV file has been processed successfully.


Expandir Datos

In [6]:
def expand_data():
    # Read the output from the previous step
    df = pd.read_csv('outputN4_Guia_FIA.csv', delimiter=';', encoding='utf-8')
    
    # First group of columns to propagate until new Asig. Par. or end marker
    course_columns = ['Asig. Par.', 'Asig. Nombre', 'Vacantes', 'Inscritos', 'Carreras']
    
    # Second group of columns to propagate until new activity/professor
    activity_columns = ['Actividad', 'RUT', 'Profesor_Nombre']
    
    # Initialize variables for current values
    current_course = {}
    current_activity = {}
    
    # Process the DataFrame row by row
    for i in range(len(df)):
        # Check if we've reached the end marker
        if df.at[i, 'Asig. Par.'] == 'Fin Guía Académica por Facultades':
            break
            
        # Process course-level information
        if pd.notna(df.at[i, 'Asig. Par.']):
            # New course found, update current values
            for col in course_columns:
                current_course[col] = df.at[i, col]
        else:
            # Propagate course information
            for col in course_columns:
                df.at[i, col] = current_course.get(col, '')
                
        # Process activity-level information
        has_new_activity = pd.notna(df.at[i, 'Actividad']) and df.at[i, 'Actividad'] != current_activity.get('Actividad', '')
        has_new_professor = pd.notna(df.at[i, 'RUT']) and df.at[i, 'RUT'] != current_activity.get('RUT', '')
        
        if has_new_activity or has_new_professor:
            # New activity or professor found, update current values
            current_activity = {}
            for col in activity_columns:
                if pd.notna(df.at[i, col]):
                    current_activity[col] = df.at[i, col]
        else:
            # Propagate activity information if we have stored values
            for col in activity_columns:
                if pd.isna(df.at[i, col]) and col in current_activity:
                    df.at[i, col] = current_activity[col]
    
    # Write the modified DataFrame to a new CSV file
    df.to_csv('outputN5_Guia_FIA.csv', sep=';', index=False, encoding='utf-8')
    
    return df

# Execute the function
if __name__ == "__main__":
    result_df = expand_data()
    print("CSV file has been processed successfully.")

CSV file has been processed successfully.


Eliminar Dia/Bloques repetidos

In [7]:

def process_schedule():
    # Read the output from the previous step
    df = pd.read_csv('outputN5_Guia_FIA.csv', delimiter=';', encoding='utf-8')
    
    # Create a copy of the DataFrame to store the processed results
    processed_df = pd.DataFrame(columns=df.columns)
    
    # Get unique combinations of Asig. Par. and RUT to identify professor blocks
    unique_blocks = df[df['RUT'].notna()][['Asig. Par.', 'Actividad', 'RUT']].drop_duplicates()
    
    # Process each block separately
    for _, block in unique_blocks.iterrows():
        # Get all rows for this professor and course
        mask = (
            (df['Asig. Par.'] == block['Asig. Par.']) & 
            (df['Actividad'] == block['Actividad']) & 
            (df['RUT'] == block['RUT'])
        )
        block_data = df[mask].copy()
        
        # Drop duplicates based on Dia and Detalle Hora within this block
        if not block_data.empty:
            # Keep only first occurrence of each Dia + Detalle Hora combination
            block_data = block_data.drop_duplicates(
                subset=['Dia', 'Detalle Hora'],
                keep='first'
            )
            
            # Append the processed block to the result
            processed_df = pd.concat([processed_df, block_data], ignore_index=True)
    
    # Add any remaining rows that weren't part of any professor block
    mask_no_prof = df['RUT'].isna()
    remaining_rows = df[mask_no_prof]
    processed_df = pd.concat([processed_df, remaining_rows], ignore_index=True)
    
    # Sort to maintain original order as much as possible
    if 'index' in processed_df.columns:
        processed_df = processed_df.sort_values('index').drop('index', axis=1)
    
    # Write the modified DataFrame to a new CSV file
    processed_df.to_csv('outputN6_Guia_FIA.csv', sep=';', index=False, encoding='utf-8')
    
    return processed_df

# Execute the function
if __name__ == "__main__":
    result_df = process_schedule()
    print("CSV file has been processed successfully.")

C:\Users\Franco\AppData\Local\Temp\ipykernel_35640\2063543194.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  processed_df = pd.concat([processed_df, block_data], ignore_index=True)


CSV file has been processed successfully.


Merch

In [8]:
import pandas as pd

def merge_duplicate_rows():
    # Read the output from the previous step
    df = pd.read_csv('outputN6_Guia_FIA.csv', delimiter=';', encoding='utf-8')
    
    # Columns to drop
    drop_columns = ['Dia', 'Blo', 'Detalle Hora']
    df = df.drop(columns=drop_columns, errors='ignore')
    
    # Group by all the specified columns, this will only merge rows that are
    # identical in all these columns
    check_columns = [
        'Asig. Par.', 'Asig. Nombre', 'Vacantes', 'Inscritos', 
        'Carreras', 'Actividad', 'RUT', 'Profesor_Nombre'
    ]
    
    # Group and sum hours
    result_df = df.groupby(
        by=check_columns,
        dropna=False
    ).agg({
        'Sala': 'first',
        'Capacidad': 'first',
        'Cantidad Horas': 'sum'
    }).reset_index()
    
    # Write the modified DataFrame to a new CSV file
    result_df.to_csv('outputN7_Guia_FIA.csv', sep=';', index=False, encoding='utf-8')
    
    return result_df

# Execute the function
if __name__ == "__main__":
    result_df = merge_duplicate_rows()
    print("CSV file has been processed successfully.")

CSV file has been processed successfully.


Chek Asigments

In [9]:
#Cuantas "Asig. Par." Unicas hay en outputN7_Guia_FIA.csv
df = pd.read_csv('outputN7_Guia_FIA.csv', delimiter=';', encoding='utf-8')
unique_courses = df['Asig. Par.'].nunique()
print(f"Unique courses: {unique_courses}")

#Cuantas "Asig. Par." Unicas hay en outputN6_Guia_FIA.csv
df = pd.read_csv('outputN6_Guia_FIA.csv', delimiter=';', encoding='utf-8')
unique_courses = df['Asig. Par.'].nunique()
print(f"Unique courses: {unique_courses}")

#Cuantas "Asig. Nombre" Unicas hay en outputN7_Guia_FIA.csv
df = pd.read_csv('outputN7_Guia_FIA.csv', delimiter=';', encoding='utf-8')
unique_courses = df['Asig. Nombre'].nunique()
print(f"Unique Asignaturas: {unique_courses}")

#Cuantas "Asig. Nombre" Unicas hay en outputN6_Guia_FIA.csv
df = pd.read_csv('outputN6_Guia_FIA.csv', delimiter=';', encoding='utf-8')
unique_courses = df['Asig. Nombre'].nunique()
print(f"Unique Asignaturas: {unique_courses}")

Unique courses: 295
Unique courses: 295
Unique Asignaturas: 175
Unique Asignaturas: 175


CSV to JSON

In [ ]:
#TODO: Revisar el campus
def extract_level_from_code(code):
    """Extract the level number from the course code."""
    if pd.isna(code) or code == '':
        return 1
        
    # Use regex to find the first number after letters
    match = re.search(r'[A-Z]+(\d)', code)
    if match:
        return int(match.group(1))
    return 1  # Default value if no number is found

def convert_csv_to_json(input_csv='outputN7_Guia_FIA.csv', output_json="../agent_input/inputOfProfesores.json"):
    # Read the CSV file
    df = pd.read_csv(input_csv, delimiter=';', encoding='utf-8')
    
    # Sort the DataFrame by RUT and Carreras
    df = df.sort_values(['RUT', 'Carreras'])
    
    # Initialize the result list and turno counter
    profesores = []
    turno_counter = 1
    current_rut = None
    
    # Group by RUT to process each professor's courses
    for rut, group in df.groupby('RUT'):
        # Skip empty RUTs
        if pd.isna(rut) or rut == '':
            continue
            
        # Get the first occurrence of professor data
        first_row = group.iloc[0]
        
        # Create profesor object
        profesor = {
            "RUT": rut,
            "Nombre": first_row['Profesor_Nombre'],
            "Turno": turno_counter,
            "Asignaturas": []
        }
        
        # Process each course for the professor
        for _, row in group.iterrows():
            asignatura = {
                "CodigoAsignatura": row['Asig. Par.'],
                "Nombre": row['Asig. Nombre'],
                "Nivel": extract_level_from_code(row['Asig. Par.']),  # Extract level from code
                "Paralelo": "A",  # Static value
                "Horas": int(row['Cantidad Horas']),
                "Vacantes": int(row['Vacantes']) if pd.notna(row['Vacantes']) else 0,
                "Campus": "Playa Brava"  # Static value
            }
            profesor["Asignaturas"].append(asignatura)
        
        profesores.append(profesor)
        turno_counter += 1
    
    # Write to JSON file
    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(profesores, f, indent=2, ensure_ascii=False)
    
    return profesores

if __name__ == "__main__":
    try:
        result = convert_csv_to_json()
        print("Conversion completed successfully!")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

Conversion completed successfully!


In [11]:
#TODO: Revisar el campus
def convert_classrooms_to_json(input_csv='outputN7_Guia_FIA.csv', output_json="../agent_input/inputOfSala.json"):
    # Read the CSV file
    df = pd.read_csv(input_csv, delimiter=';', encoding='utf-8')
    
    # Get unique classrooms and their capacities
    # Filter out empty values and 'VIRTU'
    classrooms_df = df[['Sala', 'Capacidad']].copy()
    classrooms_df = classrooms_df[
        classrooms_df['Sala'].notna() & 
        (classrooms_df['Sala'] != '') & 
        (classrooms_df['Sala'] != 'VIRTU')
    ].drop_duplicates()
    
    # Sort by classroom code
    classrooms_df = classrooms_df.sort_values('Sala')
    
    # Create the list of classrooms with sequential turno
    classrooms = []
    turno_counter = 1

    salas_kaufmann = {
        "KAUL1": True,
        "KAUS1": True,
        "KAUS2": True,
        "KAUS3": True,
        "KAUS4": True,
        "KAUS5": True,
        "KAUT1": True,
        "KAUT2": True,
        "KAUT3": True,
        "KAUT4": True,
        "KAUT5": True,
        "KAUT6": True,
        "KAUT7": True,
        "KAUT8": True,
        "KAU01": True
    }
    
    for _, row in classrooms_df.iterrows():

        # Determine campus based on classroom code
        campus = "Kaufmann" if row['Sala'] in salas_kaufmann else "Playa Brava"

        classroom = {
            "Turno": turno_counter,
            "Codigo": row['Sala'],
            "Capacidad": int(row['Capacidad']) if pd.notna(row['Capacidad']) else 0,
            "Campus": campus
        }
        classrooms.append(classroom)
        turno_counter += 1
    
    # Write to JSON file
    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(classrooms, f, indent=2, ensure_ascii=False)
    
    return classrooms

if __name__ == "__main__":
    try:
        result = convert_classrooms_to_json()
        print("Conversion completed successfully!")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

Conversion completed successfully!


Cantidad total de asignaturas

In [13]:
def count_unique_subjects(filename):
    # Read JSON file
    with open(filename, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    # Create a set to store unique subject codes
    unique_subjects = set()
    
    # Iterate through each professor and their subjects
    for profesor in data:
        for asignatura in profesor['Asignaturas']:
            unique_subjects.add(asignatura['CodigoAsignatura'])
    
    return len(unique_subjects)

# Use the function
result = count_unique_subjects('agete.json')
print(f"Número de asignaturas diferentes: {result}")

FileNotFoundError: [Errno 2] No such file or directory: 'inputOfProfesores.json'

Campus de cada Sala

In [14]:
# Diccionario de salas en Huayquique
salas_brava = {
    "AOS01": True,
    "B1": True,
    "B3": True,
    "B4": True,
    "B5": True,
    "B6": True,
    "B7": True,
    "B8": True,
    "CM3": True,
    "CM4": True,
    "CM5": True,
    "CM6": True,
    "CM8": True,
    "CPAS": True,
    "CRIQ": True,
    "CRP1": True,
    "CRP2": True,
    "CRP3": True,
    "CRP31": True,
    "CRP32": True,
    "CRP33": True,
    "CRP34": True,
    "CRP35": True,
    "CRP4": True,
    "CRP41": True,
    "CRP42": True,
    "CRP43": True,
    "CRP44": True,
    "CRP46": True,
    "CSER": True,
    "C1": True,
    "C2": True,
    "C3": True,
    "C4": True,
    "C5": True,
    "C6": True,
    "EJEMP": True,
    "ELCT": True,
    "EST01": True,
    "E1": True,
    "E10": True,
    "E11": True,
    "E2": True,
    "E3": True,
    "E4": True,
    "E5": True,
    "E6": True,
    "E7": True,
    "FAESC": True,
    "F1": True,
    "F6": True,
    "F7": True,
    "GCOV": True,
    "GESM": True,
    "GIM": True,
    "GIMC": True,
    "GIMN": True,
    "G21M": True,
    "IC1": True,
    "IC2": True,
    "IC3": True,
    "IC4": True,
    "IC8": True,
    "IES": True,
    "IM2": True,
    "IM3": True,
    "IM4": True,
    "INC1": True,
    "IOLAB": True,
    "J1": True,
    "K1": True,
    "K2": True,
    "K3": True,
    "K4": True,
    "K5": True,
    "K6": True,
    "K7": True,
    "LADU": True,
    "LAMB": True,
    "LANAQ": True,
    "LANT": True,
    "LATAS": True,
    "LAUT": True,
    "LBIG": True,
    "LBIO": True,
    "LCOA": True,
    "LCOI": True,
    "LCOM": True,
    "LCSI": True,
    "LC1": True,
    "LC2": True,
    "LC3": True,
    "LC4": True,
    "LC6": True,
    "LC7": True,
    "LC8": True,
    "LEDI1": True,
    "LEDI2": True,
    "LELE": True,
    "LELM": True,
    "LFIS": True,
    "LFLO": True,
    "LHID": True,
    "LINF": True,
    "LITI": True,
    "LMIN": True,
    "LM1": True,
    "LM2": True,
    "LOPU": True,
    "LP": True,
    "LPIR": True,
    "LPME": True,
    "LPOT": True,
    "LQ1": True,
    "LQ2": True,
    "LQ3": True,
    "LQ4": True,
    "LROB": True,
    "LRSM": True,
    "LSD": True,
    "LSO": True,
    "LSS": True,
    "MMIN": True,
    "MULT": True,
    "ODLSC": True,
    "ODPCM": True,
    "ODPOA": True,
    "ODPOB": True,
    "ODUIM": True,
    "ONLI": True,
    "ONLIN": True,
    "PLC": True,
    "PRESE": True,
    "SCAL": True,
    "SFE20": True,
    "SFH30": True,
    "SIMA": True,
    "SIMI": True,
    "SP": True,
    "SRA": True,
    "SRA01": True,
    "SRCS": True,
    "SRE": True,
    "SREC": True,
    "SRED": True,
    "SRFM": True,
    "SRI": True,
    "SRQ": True,
    "TBD": True,
    "TER": True,
    "TRT": True,
    "TSIM": True,
    "VIDQ": True,
    "VIRTU": True
}

# Cargar archivo JSON
with open("../agent_input/inputOfSala.json", 'r', encoding='utf-8') as file:
    data = json.load(file)

# Inicializar contadores
salas_encontradas = 0
salas_no_encontradas = 0

# Contar matches y no-matches
for sala in data:
    codigo = sala.get("Codigo", "")
    if codigo in salas_brava:
        salas_encontradas += 1
    else:
        salas_no_encontradas += 1

print(f"Salas encontradas: {salas_encontradas}")
print(f"Salas no encontradas: {salas_no_encontradas}")


Salas encontradas: 45
Salas no encontradas: 28


In [15]:
# Diccionario de salas en Huayquique
salas_kaufmann = {
    "KAUL1": True,
    "KAUS1": True,
    "KAUS2": True,
    "KAUS3": True,
    "KAUS4": True,
    "KAUS5": True,
    "KAUT1": True,
    "KAUT2": True,
    "KAUT3": True,
    "KAUT4": True,
    "KAUT5": True,
    "KAUT6": True,
    "KAUT7": True,
    "KAUT8": True,
    "KAU01": True
}

# Cargar archivo JSON
with open("../agent_input/inputOfSala.json", 'r', encoding='utf-8') as file:
    data = json.load(file)

# Inicializar contadores
salas_encontradas = 0
salas_no_encontradas = 0

# Contar matches y no-matches
for sala in data:
    codigo = sala.get("Codigo", "")
    if codigo in salas_kaufmann:
        salas_encontradas += 1
    else:
        salas_no_encontradas += 1

print(f"Salas encontradas: {salas_encontradas}")
print(f"Salas no encontradas: {salas_no_encontradas}")


Salas encontradas: 13
Salas no encontradas: 60


In [16]:
# Diccionario de salas en Huayquique
salas_huayquique = {
    "HLA1": True, "HLA2": True, "HLA3": True, "H10": True, "H12": True,
    "H13": True, "H14": True, "H16": True, "H17": True, "H18": True,
    "H20": True, "H21": True, "H22": True, "H4": True, "H5": True,
    "H6": True, "LBDE": True, "LCHUA": True, "LCUL": True, "LECO": True,
    "LFVE": True, "LGEN": True, "LINV": True, "LMIC": True, "LOCE": True,
    "LPEC": True, "LZOO": True, "SRAH": True
}

# Cargar archivo JSON
with open("../agent_input/inputOfSala.json", 'r', encoding='utf-8') as file:
    data = json.load(file)

# Inicializar contadores
salas_encontradas = 0
salas_no_encontradas = 0

# Contar matches y no-matches
for sala in data:
    codigo = sala.get("Codigo", "")
    if codigo in salas_huayquique:
        salas_encontradas += 1
    else:
        salas_no_encontradas += 1

print(f"Salas encontradas: {salas_encontradas}")
print(f"Salas no encontradas: {salas_no_encontradas}")


Salas encontradas: 0
Salas no encontradas: 73


In [17]:
# Diccionario de salas en Huayquique
salas_lynch = {
    "BIOM": True,
    "EK01": True,
    "EK02": True,
    "EK03": True,
    "IRER": True,
    "LBI2": True,
    "LFEJ": True,
    "LFST": True,
    "LGIT": True
}


# Cargar archivo JSON
with open("../agent_input/inputOfSala.json", 'r', encoding='utf-8') as file:
    data = json.load(file)

# Inicializar contadores
salas_encontradas = 0
salas_no_encontradas = 0

# Contar matches y no-matches
for sala in data:
    codigo = sala.get("Codigo", "")
    if codigo in salas_lynch:
        salas_encontradas += 1
    else:
        salas_no_encontradas += 1

print(f"Salas encontradas: {salas_encontradas}")
print(f"Salas no encontradas: {salas_no_encontradas}")


Salas encontradas: 0
Salas no encontradas: 73


In [18]:
# Diccionario de salas en Huayquique
salas_canchones = {
    "CANC": True,
    "CANH": True,
    "CAN1": True,
    "CAN2": True,
    "CAN3": True,
    "CAN4": True,
    "CAN5": True,
    "TERR": True
}

# Cargar archivo JSON
with open("../agent_input/inputOfSala.json", 'r', encoding='utf-8') as file:
    data = json.load(file)

# Inicializar contadores
salas_encontradas = 0
salas_no_encontradas = 0

# Contar matches y no-matches
for sala in data:
    codigo = sala.get("Codigo", "")
    if codigo in salas_canchones:
        salas_encontradas += 1
    else:
        salas_no_encontradas += 1

print(f"Salas encontradas: {salas_encontradas}")
print(f"Salas no encontradas: {salas_no_encontradas}")


Salas encontradas: 0
Salas no encontradas: 73


In [19]:
# Diccionario de salas en Huayquique
salas_astoreca = {
    "ASTO1": True,
    "ASTO2": True,
    "ASTO3": True,
    "ASTO4": True,
    "ASTO5": True
}

# Cargar archivo JSON
with open("../agent_input/inputOfSala.json", 'r', encoding='utf-8') as file:
    data = json.load(file)

# Inicializar contadores
salas_encontradas = 0
salas_no_encontradas = 0

# Contar matches y no-matches
for sala in data:
    codigo = sala.get("Codigo", "")
    if codigo in salas_astoreca:
        salas_encontradas += 1
    else:
        salas_no_encontradas += 1

print(f"Salas encontradas: {salas_encontradas}")
print(f"Salas no encontradas: {salas_no_encontradas}")


Salas encontradas: 0
Salas no encontradas: 73
